In [20]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [21]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [22]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [23]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 17772.47it/s]


In [24]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 128.94it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [25]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [26]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [27]:
data = kernelclass.calc_kernel_mtxel()

In [28]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.47881-0.00000j | -0.02994-0.11418j | -0.07084-0.02386j | -0.06933+0.12470j | -0.10282+0.15727j | 0.01616-0.05548j | 0.02306+0.09088j | 0.00050-0.00007j
-0.02994-0.11418j | 0.21656+0.00000j | 0.02322-0.03526j | -0.01349-0.01293j | 0.02786+0.03082j | -0.01573-0.00945j | -0.10018-0.00079j | -0.00211+0.01527j
-0.07084-0.02386j | 0.02322-0.03526j | 0.21656+0.00000j | 0.00335-0.01821j | -0.01061+0.04092j | 0.00415+0.10010j | 0.01008+0.01557j | 0.00049-0.00024j
-0.06933+0.12470j | -0.01349-0.01293j | 0.00335-0.01821j | 0.24228-0.00000j | 0.09993+0.00717j | 0.00642-0.00146j | -0.00472+0.00445j | 0.05010+0.06913j
-0.10282+0.15727j | 0.02786+0.03082j | -0.01061+0.04092j | 0.09993+0.00717j | 0.21656+0.00000j | -0.01754+0.00533j | 0.01264-0.01377j | -0.00966-0.01149j
0.01616-0.05548j | -0.01573-0.00945j | 0.00415+0.10010j | 0.00642-0.00146j | -0.01754+0.00533j | 0.24235+0.00000j | -0.00570+0.00336j | 0.00109+0.00256j
0.02306+0.09088j | -0.10018-0.00079j | 0.01008+0.01557j | -0

In [29]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.05016-0.19131j | -0.11326-0.03814j | -0.12329+0.21733j | -0.17991+0.27518j | 0.02843-0.09821j | 0.04308+0.15990j | -0.00000+0.00000j
-0.05016-0.19131j | 1.00000+0.00000j | 0.04481-0.06806j | -0.24097-0.23319j | 0.05318+0.05884j | -0.28182-0.17010j | -0.03101+0.00090j | -0.00024+0.00175j
-0.11326-0.03814j | 0.04481-0.06806j | 1.00000-0.00000j | 0.06072-0.32644j | -0.02049+0.07906j | 0.00142+0.03097j | 0.18195+0.27839j | 0.00017-0.00008j
-0.12329+0.21733j | -0.24097-0.23319j | 0.06072-0.32644j | 1.00000-0.00000j | 0.03087+0.00288j | -0.04309+0.00855j | 0.03204-0.02999j | 0.01672+0.02048j
-0.17991+0.27518j | 0.05318+0.05884j | -0.02049+0.07906j | 0.03087+0.00288j | 1.00000+0.00000j | -0.31441+0.09672j | 0.22525-0.24866j | -0.00121-0.00144j
0.02843-0.09821j | -0.28182-0.17010j | 0.00142+0.03097j | -0.04309+0.00855j | -0.31441+0.09672j | 1.00000+0.00000j | 0.03747-0.02293j | 0.00345-0.00052j
0.04308+0.15990j | -0.03101+0.00090j | 0.18195+0.27839j | 0.03204-0.02999j | 0

In [30]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | -0.03187-0.12155j | -0.07195-0.02423j | 0.02096-0.03695j | -0.11429+0.17483j | -0.00484+0.01670j | -0.00732-0.02719j | -0.00000+0.00000j
-0.03187-0.12155j | 0.00000+0.00000j | 0.00750-0.01138j | -0.11628-0.11212j | 0.00890+0.00985j | -0.13604-0.08147j | 0.00650-0.00020j | -0.00011+0.00079j
-0.07195-0.02423j | 0.00750-0.01138j | 0.00000+0.00000j | 0.02952-0.15721j | -0.00343+0.01322j | -0.00029-0.00650j | 0.08784+0.13399j | 0.00004-0.00002j
0.02096-0.03695j | -0.11628-0.11212j | 0.02952-0.15721j | 0.00000+0.00000j | -0.00647-0.00063j | -0.00130+0.00026j | 0.00097-0.00091j | -0.00158-0.00282j
-0.11429+0.17483j | 0.00890+0.00985j | -0.00343+0.01322j | -0.00647-0.00063j | 0.00000+0.00000j | -0.15163+0.04607j | 0.10869-0.11963j | -0.00051-0.00061j
-0.00484+0.01670j | -0.13604-0.08147j | -0.00029-0.00650j | -0.00130+0.00026j | -0.15163+0.04607j | 0.00000+0.00000j | 0.00113-0.00069j | 0.00144-0.00072j
-0.00732-0.02719j | 0.00650-0.00020j | 0.08784+0.13399j | 0.00097-0.00091

In [31]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.26829+0.00000j | -0.09855-0.37586j | -0.22032-0.07421j | -0.20946+0.37986j | -0.35675+0.54570j | 0.04916-0.16834j | 0.06834+0.27634j | 0.01063-0.00128j
-0.09855-0.37586j | 0.12974-0.00000j | 0.16075-0.24413j | -0.30650-0.28890j | 0.19212+0.21255j | -0.35483-0.21229j | -0.07330-0.00212j | -0.00980+0.07101j
-0.22032-0.07421j | 0.16075-0.24413j | 0.12974+0.00000j | 0.07330-0.41063j | -0.07345+0.28342j | 0.00284+0.07327j | 0.22336+0.35309j | 0.00243-0.00116j
-0.20946+0.37986j | -0.30650-0.28890j | 0.07330-0.41063j | 0.04804-0.00000j | 0.07319+0.00438j | 0.15222-0.03334j | -0.11334+0.10668j | 0.03408+0.05245j
-0.35675+0.54570j | 0.19212+0.21255j | -0.07345+0.28342j | 0.07319+0.00438j | 0.12974+0.00000j | -0.39591+0.11839j | 0.28907-0.30673j | -0.04502-0.05359j
0.04916-0.16834j | -0.35483-0.21229j | 0.00284+0.07327j | 0.15222-0.03334j | -0.39591+0.11839j | 0.04796-0.00000j | -0.13392+0.07982j | -0.00712+0.00528j
0.06834+0.27634j | -0.07330-0.00212j | 0.22336+0.35309j | -0.11334+0.10668j | 

Kernelv2

In [32]:
from kernelv2 import KernelMtxElv2

kernelclassv2 = KernelMtxElv2.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [33]:
datav2 = kernelclassv2.kernel_mtxel()

In [34]:
exc = datav2["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.47881-0.00000j | -0.02994+0.11418j | -0.07084+0.02386j | -0.06933-0.12470j | -0.10282-0.15727j | 0.01616+0.05548j | 0.02306-0.09088j | 0.00050+0.00007j
-0.02994-0.11418j | 0.21656+0.00000j | 0.02322+0.03526j | -0.01349+0.01293j | 0.02786-0.03082j | -0.01573+0.00945j | -0.10018+0.00079j | -0.00211-0.01527j
-0.07084-0.02386j | 0.02322-0.03526j | 0.21656+0.00000j | 0.00335+0.01821j | -0.01061-0.04092j | 0.00415-0.10010j | 0.01008-0.01557j | 0.00049+0.00024j
-0.06933+0.12470j | -0.01349-0.01293j | 0.00335-0.01821j | 0.24228-0.00000j | 0.09993-0.00717j | 0.00642+0.00146j | -0.00472-0.00445j | 0.05010-0.06913j
-0.10282+0.15727j | 0.02786+0.03082j | -0.01061+0.04092j | 0.09993+0.00717j | 0.21656+0.00000j | -0.01754-0.00533j | 0.01264+0.01377j | -0.00966+0.01149j
0.01616-0.05548j | -0.01573-0.00945j | 0.00415+0.10010j | 0.00642-0.00146j | -0.01754+0.00533j | 0.24235+0.00000j | -0.00570-0.00336j | 0.00109-0.00256j
0.02306+0.09088j | -0.10018-0.00079j | 0.01008+0.01557j | -0

In [35]:
head = datav2["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.05016+0.19131j | -0.11326+0.03815j | -0.11955-0.21940j | -0.17990-0.27519j | 0.02876+0.09811j | 0.03840-0.16109j | -0.00000-0.00000j
-0.05016-0.19131j | 1.00000+0.00000j | 0.04481+0.06806j | -0.24317+0.23088j | 0.05318-0.05883j | -0.28255+0.16888j | -0.03099+0.00139j | -0.00024-0.00175j
-0.11326-0.03814j | 0.04481-0.06806j | 1.00000-0.00000j | 0.05944+0.32668j | -0.02049-0.07906j | 0.00114-0.03098j | 0.17941-0.28003j | 0.00017+0.00008j
-0.12329+0.21733j | -0.24097-0.23319j | 0.06072-0.32644j | 1.00000-0.00000j | 0.03097-0.00156j | -0.04255-0.01090j | 0.03180+0.03024j | 0.01426-0.02226j
-0.17991+0.27518j | 0.05318+0.05884j | -0.02049+0.07906j | 0.03087+0.00288j | 1.00000+0.00000j | -0.31508-0.09453j | 0.22864+0.24555j | -0.00121+0.00144j
0.02843-0.09821j | -0.28182-0.17010j | 0.00142+0.03097j | -0.04309+0.00855j | -0.31441+0.09672j | 1.00000+0.00000j | 0.03819+0.02171j | -0.00276-0.00213j
0.04308+0.15990j | -0.03101+0.00090j | 0.18195+0.27839j | 0.03204-0.02999j | 

In [36]:
wings = datav2["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | -0.02333+0.08898j | -0.05267+0.01774j | 0.00326+0.00598j | -0.08366-0.12797j | -0.00079-0.00267j | -0.00105+0.00439j | -0.00000-0.00000j
-0.02333-0.08898j | 0.00000+0.00000j | 0.00232+0.00352j | -0.11858+0.11375j | 0.00279-0.00309j | -0.13811+0.08333j | 0.04011-0.00100j | -0.00030-0.00214j
-0.05267-0.01774j | 0.00232-0.00352j | 0.00000+0.00000j | 0.02920+0.16007j | -0.00106-0.00408j | -0.00158+0.04008j | 0.08831-0.13697j | 0.00007+0.00003j
0.00336-0.00592j | -0.11867-0.11366j | 0.02968-0.15999j | 0.00000+0.00000j | -0.04004+0.00247j | -0.00204-0.00052j | 0.00153+0.00145j | -0.01148+0.01606j
-0.08366+0.12797j | 0.00279+0.00309j | -0.00106+0.00408j | -0.03998-0.00327j | 0.00000+0.00000j | -0.15416-0.04714j | 0.11123+0.12108j | -0.00135+0.00161j
-0.00077+0.00268j | -0.13844-0.08277j | -0.00174-0.04008j | -0.00207+0.00041j | -0.15433+0.04658j | 0.00000+0.00000j | 0.00183+0.00104j | -0.00049+0.00048j
-0.00117-0.00436j | 0.04012-0.00036j | 0.08877+0.13667j | 0.00154-0.0014

In [37]:
body = datav2["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.26817-0.00000j | -0.08153+0.31100j | -0.18194+0.06127j | -0.20934-0.37247j | -0.29583-0.45250j | 0.04821+0.16606j | 0.07117-0.27145j | 0.01449+0.00175j
-0.08153-0.31100j | 0.12970+0.00000j | 0.16278+0.24723j | -0.30883+0.29937j | 0.19464-0.21534j | -0.36171+0.21783j | -0.03912-0.00074j | -0.01049-0.07604j
-0.18194-0.06129j | 0.16278-0.24722j | 0.12970-0.00000j | 0.07857+0.41863j | -0.07438-0.28700j | 0.00175-0.03908j | 0.23426-0.35654j | 0.00258+0.00123j
-0.20592+0.37437j | -0.31217-0.29588j | 0.07557-0.41918j | 0.04805+0.00000j | 0.03898-0.00339j | 0.14062+0.03321j | -0.10485-0.09916j | 0.02133-0.02572j
-0.29583+0.45251j | 0.19464+0.21534j | -0.07437+0.28700j | 0.03906+0.00221j | 0.12970-0.00000j | -0.40338-0.12386j | 0.28837+0.31948j | -0.04819+0.05735j
0.04851-0.16597j | -0.36221-0.21699j | 0.00148+0.03909j | 0.14116-0.03079j | -0.40411+0.12147j | 0.04797+0.00000j | -0.12487-0.07282j | 0.00588+0.00131j
0.06688+0.27254j | -0.03911-0.00136j | 0.22927+0.35977j | -0.10509+0.09891j | 0